In [65]:
import music21 as m21
import re
from collections import Counter
from swalign_local import *

In [267]:
# Step 1: Read input data:
import music21 as m21
import re
from collections import Counter
from swalign_local import *
import csv
import glob

#score_path = "jkupddtest/bachmono/wtc2f20.krn"
#score_path = "jkupddtest/beethovenmono/sonata01-3.krn"
#score_path = "jkupddtest/mozartmono/sonata04-2.krn"
#score_path = "jkupddtest/gibbonsmono/silverswan.krn"
score_path = "jkupddtest/chopinmono/mazurka24-4.krn"
#score_path = '~/facets-search-engine/data/Beethoven9thOdeToJoy.xml'

raw_score = m21.converter.parse(score_path)
num_of_parts = len(raw_score.parts)

In [268]:
num_of_parts

2

In [269]:
voices = []
for part in raw_score.parts:
    voices.append(part)

In [270]:
# Step 2: Get diatonic scale degree sequence from all voices

def get_diatonic_intervals(voice):
    
    # get key / root information:
    roots = [key.tonic for key in voice.recurse().getElementsByClass(m21.key.Key)]
    diatonic_root = roots[0].diatonicNoteNum if roots != [] else voice.analyze('key').tonic.diatonicNoteNum
    #print(diatonic_root)
    key = voice.analyze('key')

    noteoffset = []
    midi = []
    #timesig = m21_score.getContextByClass(m21.meter.TimeSignature)
    #barlength = timesig.barduration

    dict_wordtonum = {"Unison": '0', "Second": '1', "Third": '2', "Fourth": '3', "Fifth": '4', "Sixth": '5', "Seventh": '6'}
        
    dict_encode_dia_intervals = {"6D": 'A', '5D': 'B', '4D': 'C', '3D':'D', '2D':'E', '1D':'F', 
                                     '1A':'G', '2A':'H', '3A':'I', '4A':'J', '5A': 'K', '6A':'L', '0A': 'M'}
        
    diatonic_intervals = []
    beatstrength = []
    previous_note = None

    onset = 0
    for thisnote in voice.recurse().notes:
        onset += thisnote.offset
        
        # We ignore rests
        if thisnote.isRest: 
                # If the rest is a full measure, part of a multi-measure rest: we need to adjust
                continue
        
        noteoffset.append(onset)
        beatstrength.append(thisnote.beatStrength)
        #if thisnote.isNote:
        #    midi.append(thisnote.pitch.midi)
        #elif thisnote.isChord:
        #    midi.append(thisnote.root().midi)
            
        if previous_note is None:
                previous_note = thisnote
        else:
                if thisnote.isNote and previous_note.isNote:
                    # gap = number of semi-tones of the current interval 
                    gap = thisnote.pitch.diatonicNoteNum - previous_note.pitch.diatonicNoteNum
                elif thisnote.isChord and previous_note.isNote:
                    gap = thisnote.root().diatonicNoteNum - previous_note.pitch.diatonicNoteNum
                elif thisnote.isChord and previous_note.isChord:
                    gap = thisnote.root().diatonicNoteNum - previous_note.root().diatonicNoteNum
                elif thisnote.isNote and previous_note.isChord:
                    gap = thisnote.pitch.diatonicNoteNum - previous_note.root().diatonicNoteNum
                # if a pitch change is detected
                    
                if gap != 0:
                    
                    if gap > 0:
                        #  if the semi-tone difference between the current and the previous item > 0, it is an ascending interval.
                        direction = 'A'
                    else:
                        #  otherwise, it is a descending interval.
                        direction = 'D'

                    # Get intervals using music21
                    """
                            "directedSimpleNiceName" examples: "Descending Doubly-Diminished Fifth", "Ascending Perfect Fourth", "Ascending Doubly-Augmented Fourth"
                            "simpleName" examples: dd5, P5, AA4. There's no direction information
                            Since it only executes when a pitch interval is detected, "unison" refers to an augmented unison, a.k.a minor second
                    """
                    # take intervals between root notes if there exists any chord
                    if previous_note.isChord:
                        startnote = previous_note.root()
                    else:
                        startnote = previous_note
                    if thisnote.isChord:
                        endnote = thisnote.root()
                    else:
                        endnote = thisnote
                        
                    m21_interval_directed = m21.interval.Interval(noteStart=startnote, noteEnd=endnote).directedSimpleNiceName

                    arr_diatonic = m21_interval_directed.split(" ")

                    m21_generic = dict_wordtonum[arr_diatonic[-1]]
                    
                    # m21_interval: 2A, 3D etc...
                    m21_interval = m21_generic+direction
                    if m21_generic == "0":
                        m21_interval = '0A'
                    # to make each m21_interval unique, show direction and each as single character in string, we encode the diatonic intervals as letters
                    encode_interval = dict_encode_dia_intervals[m21_interval]
                    diatonic_intervals.append(encode_interval)

                else:
                    # We take the interval between two consecutive pitches as 0A
                    encode_interval = dict_encode_dia_intervals['0A']
                    diatonic_intervals.append(encode_interval)
                previous_note = thisnote
            
    string = ""
    for i in diatonic_intervals:
        string += str(i)
        
    return string, key, noteoffset, midi, beatstrength

In [271]:
strings = []
noteoffsets = []
midis = []
keys = []
beatstrengths = []

for voice in voices:
    string, key, noteoffset, midi, beatstrength = get_diatonic_intervals(voice)
    strings.append(string)
    noteoffsets.append(noteoffset)
    beatstrengths.append(beatstrength)
    midis.append(midi)
    keys.append(key)
    #print(len(string), len(beatstrength))#, len(midi))
    #print(string)
    #print(len(noteoffset))
    #print(midi)
    #print(beatstrength)
    

In [272]:
# Smith-Waterman alignment between the two input sequences

def swm_alignment(string1, string2):
    match = 2
    mismatch = -2
    scoring = NucleotideScoringMatrix(match, mismatch)
    sw = LocalAlignment(scoring) 
    
    alignment = sw.align(string1, string2)
    alignment_strings = alignment.dump()

    simi_score = alignment.matches / (alignment.mismatches + alignment.matches)
    
    return alignment_strings, simi_score


In [273]:
def filter_and_gather_patterns(allthepatterns, fullseq):
    pat = fullseq.split('-')
    print("num of pat in this seq:", len(pat))    
    for pattern in pat:
        # Get rid of the sequence shorter than 3 intervals; put all alignment patterns into all pattern list
        if len(pattern) > 2:
            allthepatterns.append(pattern)
    return allthepatterns

In [274]:
# compare each pair of voices!
allthepatterns = []

for i in range(0, len(voices)-1):
    for j in range(i+1, len(voices)):
        alignment_strings, simi_score = swm_alignment(strings[i], strings[j])
        if simi_score < 0.25:
            # if two voices are too different, getting patterns from their alignment won't make sense
            print("ignoring voice pair:", i, j)
            continue
        # get the patterns from the first voice of this pair
        fullseq1 = alignment_strings[0]
        fullseq2 = alignment_strings[1]
        allthepatterns = filter_and_gather_patterns(allthepatterns, fullseq1)
        allthepatterns = filter_and_gather_patterns(allthepatterns, fullseq2)
        

Query:  431 MEIMDIMD-GMFKMBG--MGMEMGMFMMMIMDIMD-GMFKMBI-MDHMEJMC-MM-M-JM-MMMCIMDM-M-MJM--M-M-M---CKM-M--MIFHMM-MMBM-MEM--MEM-MMMM 521
            | |.|| | | |   |  .|| || |..||.|| | | |...| |  |   | || | .| ||| .| | | |.|  | | |   | | |  |   || ||.| |||  ||| ||||
Ref  :  913 M-ILDI-DEG-F---GFFIGM-MG-FCFMILDI-DEG-FGFFIGM--M---CFMMFMAGMGMMM-GM-MFMFMEMIFMEMFMJFMC-MFMKGM---MMGMMFMFMEMIFMEMFMMMM 1008

num of pat in this seq: 27
num of pat in this seq: 22


In [275]:
allthepattern_count = dict(Counter(allthepatterns))

# get a pattern set P of patterns p
allthepatterns = list(set(allthepatterns))

In [276]:
len(allthepatterns)

21

In [277]:
def decode_strings(string):
    
    decode_dia_intervals = {'A': '6D', 'B': '5D', 'C': '4D', 'D': '3D', 'E': '2D', 'F': '1D', 
                            'G': '1A', 'H': '2A', 'I': '3A', 'J': '4A', 'K': '5A', 'L': '6A', 'M': '0A'}
    
    decode_string = ""

    for letter in string:
        trans = decode_dia_intervals[letter]
        decode_string+=trans

    return decode_string

In [278]:
def printLCSSubStr(X: str, Y: str,
                   m: int, n: int):
 
    # Create a table to store lengths of longest common suffixes of substrings.
    # Note that LCSuff[i][j] contains length of longest common suffix of X[0..i-1] and
    # Y[0..j-1]. The first row and first column entries have no logical meaning,
    # they are used only for simplicity of program
    LCSuff = [[0 for i in range(n + 1)]
                 for j in range(m + 1)]
 
    # To store length of the longest common substring
    length = 0
 
    # To store the index of the cell which contains the maximum value.
    # This cell's index helps in building up the longest common substring from right to left.
    row, col = 0, 0
 
    # Following steps build LCSuff[m+1][n+1]
    # in bottom up fashion.
    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0 or j == 0:
                LCSuff[i][j] = 0
            elif X[i - 1] == Y[j - 1]:
                LCSuff[i][j] = LCSuff[i - 1][j - 1] + 1
                if length < LCSuff[i][j]:
                    length = LCSuff[i][j]
                    row = i
                    col = j
            else:
                LCSuff[i][j] = 0
 
    # if true, then no common substring exists
    if length == 0:
        #print("No common substring")
        return ""
 
    # allocate space for the longest common substring
    resultStr = ['0'] * length
 
    # traverse up diagonally form the (row, col) cell until LCSuff[row][col] != 0
    while LCSuff[row][col] != 0:
        length -= 1
        resultStr[length] = X[row - 1] # or Y[col-1]
        # move diagonally up to previous cell
        row -= 1
        col -= 1
 
    # required longest common substring
    return ''.join(resultStr) 


The following function returns the longest substring that appeared at least once.

So, if there is length 4 substring repeated 4 times and a length 7 substring repeated 2 times, we take the length 7.
        

In [279]:
def reduce_pattern_length(seq):
        best_performance = ""
        candidates = []
        for length in range(4, int(len(seq)*0.5)+1): 
            for start in range(0, len(seq)-length):
                # get all the substrings of this length within the string, save in candidates
                candidates.append(seq[start:start+length])
        count_can = {}
        allthepattern_count = dict(Counter(allthepatterns))

        for candidate in candidates:
                count_can[candidate] = seq.count(candidate)
                if count_can[candidate] > 1:
                    # if it is repeated more than once in the string
                    if len(candidate) > len(best_performance):
                        best_performance = candidate
                    elif len(candidate) == len(best_performance) and count_can[candidate] > count_can[best_performance]:
                        best_performance = candidate
        
        if best_performance == "":
            return "", 0
        return best_performance, count_can[best_performance]
    
def check_reduction(item):
    success = False
    frequent_substr, timesitappear = reduce_pattern_length(item)
    if len(frequent_substr) > 4:
        #print(frequent_substr, item, timesitappear)
        if len(frequent_substr)*timesitappear >= 0.6*len(item):
            print("extracted", frequent_substr, "from", item)
            # sucessfully reduced the pattern
            return frequent_substr
    return None

In [280]:
def reduce_length_of_each_pattern(patterns_to_reduce, allthepattern_count):
    beforereduction = {}
    # reducedfinalpat is the new list that contains all valid patterns after reduction plus the ones that does not need to be reduced
    reducedfinalpat = []
    extracount_reducedpat = {}

    for item in patterns_to_reduce:
        # check the ones that has substring, see if their length can be reduced
        if len(item) > 11:
            #if the extracted pattern is too long, find longest repeated pattern within it
            frequent_substr = check_reduction(item)
            if frequent_substr != None:
                if len(frequent_substr) > 23:
                    # second round, if a frequent substring is takend and it is still quite long
                    new_frequent_substr = check_reduction(frequent_substr)
                    if new_frequent_substr != None:
                        # if the second attempt is a success, take the twice reduced pattern
                        if new_frequent_substr not in extracount_reducedpat:
                            # this extracted pattern should have an extra count: 
                            # the times it appeared in item * times the item appeared in alignment segmentation
                            extracount_reducedpat[new_frequent_substr] = allthepattern_count[item] * item.count(new_frequent_substr)
                        else:
                            #it has already been a reduction of another pattern, just add
                            extracount_reducedpat[new_frequent_substr] += allthepattern_count[item] * item.count(new_frequent_substr)
                        reducedfinalpat.append(new_frequent_substr)
                        beforereduction[new_frequent_substr] = item
                    else:
                        # if the second attempt failed, take the result of first reduction
                        if frequent_substr not in extracount_reducedpat:
                            # this extracted pattern should have an extra count: 
                            # the times it appeared in item * times the item appeared in alignment segmentation
                            extracount_reducedpat[frequent_substr] = allthepattern_count[item] * item.count(frequent_substr)
                        else:
                            #it has already been a reduction of another pattern, just add
                            extracount_reducedpat[frequent_substr] += allthepattern_count[item] * item.count(frequent_substr) 
                        reducedfinalpat.append(frequent_substr)
                        beforereduction[frequent_substr] = item
                else:
                    # if the extracted pattern is already not that long
                    if frequent_substr not in extracount_reducedpat:
                            # this extracted pattern should have an extra count: 
                            # the times it appeared in item * times the item appeared in alignment segmentation
                            extracount_reducedpat[frequent_substr] = allthepattern_count[item] * item.count(frequent_substr)
                    else:
                            #it has already been a reduction of another pattern, just add
                            extracount_reducedpat[frequent_substr] += allthepattern_count[item] * item.count(frequent_substr) 
                    reducedfinalpat.append(frequent_substr)
                    beforereduction[frequent_substr] = item
            else:
                # if the reduction failed, take the original
                reducedfinalpat.append(item)
        else:
            # otherwise keep it without reduction
            reducedfinalpat.append(item)
    
    return reducedfinalpat, beforereduction, extracount_reducedpat

In [281]:
reducedpat, beforereduction, extracount_reducedpat = reduce_length_of_each_pattern(allthepatterns, allthepattern_count)

In [282]:
def count_final_pattern(reducedfinalpat, strings, beatstrengths):
    # Analyze patterns by the beat strength of the beginning note of patterns, 
    # and check if they have repeatedly appeared in the whole piece

    count_pattern = {}
    refined_finalpat = []
    startonsets = []
    startpitches = []
    endonsets = []
    endpitches = []

    for pattern in reducedfinalpat:
        stringnum = 0
        beat_importance = 0
        for string in strings:
            # find the pattern in each voice
            patpos_start, patpos_end, count, beatcount = find_pattern_pos(string, pattern, beatstrengths[stringnum])
            beat_importance += beatcount
            if pattern not in count_pattern:
                count_pattern[pattern] = count
            else:
                count_pattern[pattern] += count
            if count != 0:
                print("Pattern", pattern, "appears in voice no.", stringnum+1, ", first pos starts at ", patpos_start[0], "end at", patpos_end[0])
            num = 0
            for startpos in patpos_start:
                startonsets.append(noteoffsets[stringnum][startpos])
                endonsets.append(noteoffsets[stringnum][patpos_end[num]])
                num+=1
            stringnum+=1
        if count_pattern[pattern] >= 2 and beat_importance >= 1:
            # get rid of the ones that actually appeared less than 2 times in total
            refined_finalpat.append(pattern)
        else:
            if count_pattern[pattern] < 2:
                print("discard pattern", pattern, "because it only appered once in total.")
            elif beat_importance == 0:
                print("discard pattern", pattern, "because it never appeared at the start of a bar.")
                print("this pattern can be decoded as", decode_strings(pattern))
            
    return refined_finalpat, count_pattern, startonsets, endonsets


In [283]:
#dict(Counter(reducedpat))


In [284]:
# originally, reduced_pattern_count is how many times it appeared in the extraction
# but it shouldn't be counted like this

# we modify it to occ(p) to count how many times the reduced pattern appeared in the alignment+length reduction process
# for the ones that are reduced, we call the pattern before reduction an original pattern, then its occ should be:
# times it appeared in the original pattern*occ(original pattern)+how many times it appeared as original pattern
reduced_pattern_count = {}
for pattern in reducedpat:
    # initialize
    reduced_pattern_count[pattern] = 0    
    if pattern in beforereduction:
        if pattern in allthepattern_count:
            reduced_pattern_count[pattern] = allthepattern_count[pattern]
        # if it was reduced, then it should be extra counted here
        reduced_pattern_count[pattern] += extracount_reducedpat[pattern]
    else:
        # if it's not reduced, then it keep the count as the count after segmentation
        reduced_pattern_count[pattern] = allthepattern_count[pattern]

In [285]:
#reduced_pattern_count

Get longest common substring between two long patterns

In [286]:
lcscount = {}
dictoflcs = {}
for i in range(0, len(reducedpat)-1):
    for j in range(i+1, len(reducedpat)):
        #print(i, j)
        pat1 = reducedpat[i]
        pat2 = reducedpat[j]
        if len(pat1) > 11 and len(pat2) > 11 and pat1 != pat2:
            lcs = printLCSSubStr(pat1, pat2, len(pat1), len(pat2))
            if len(lcs) >= 4 and lcs != pat1 and lcs != pat2: 
                # if lcs is not one of themselves. 
                # we are not interested in longest common substrings that are already in the pattern set
                # because substring relationship will be investigated in DAG
                if pat1 not in dictoflcs:
                    dictoflcs[pat1] = []
                if pat2 not in dictoflcs:
                    dictoflcs[pat2] = []
                dictoflcs[pat1].append(lcs)
                dictoflcs[pat2].append(lcs)
                if lcs in lcscount:
                    lcscount[lcs] +=1
                else:
                    lcscount[lcs] = 1

newdict = {}
# print out the common strings that appeared more than 3 times
for temp in lcscount:
    if lcscount[temp] >= 2:
        if temp not in reducedpat:
            newdict[temp] = lcscount[temp]
        #print(temp, decode_strings(temp), lcscount[temp])


In [287]:
newlist = list(newdict)

listofrelated = {}
for i in range(0, len(newlist)-1):
    for j in range(i+1, len(newlist)):
        if newlist[i] in newlist[j] or newlist[j] in newlist[i]:
            if newlist[i] not in listofrelated:
                listofrelated[newlist[i]] = []
            if newlist[j] not in listofrelated:
                listofrelated[newlist[j]] = []
            listofrelated[newlist[i]].append(newlist[j])
            listofrelated[newlist[j]].append(newlist[i])

# recude one cluster into one
final_lcs = []
for candidate in newlist:
    mostcounted = candidate
    if candidate in listofrelated:
        for related in listofrelated[candidate]:
            if lcscount[related] > lcscount[mostcounted]:# discarded: and abs(len(related)-len(candidate)) <= 4: # if it is counted more and not way too short or longer than the candidate
                mostcounted = related
            elif lcscount[related] == lcscount[mostcounted] and len(related) > len(mostcounted):# discarded: and abs(len(related)-len(candidate)) <= 4:
                # take the longer one if the count is the same
                mostcounted = related
    if mostcounted not in final_lcs:
        final_lcs.append(mostcounted)

In [288]:
def find_pattern_pos(string, pattern, beatstrength):
    patpos_start = []
    patpos_end = []
    count = 0
    beatcount = 0
    for m in re.finditer(pattern, string):
        count +=1
        if beatstrength[m.start()] == 1 or m.start() <= 8: # if it starts at a strong beat or is in the beginning 8 notes of a voice
            beatcount += 1
        patpos_start.append(m.start())
        patpos_end.append(m.end())
    return patpos_start, patpos_end, count, beatcount

In [289]:
# The list of LCS patterns, not filtered with beat strength
#print(final_lcs)
#for i in final_lcs:
#    print(decode_strings(i))

In [290]:
# Filtered by beat strength:
testpat, count_pattern_test, startonsetstest,  endonsetstest = count_final_pattern(final_lcs, strings, beatstrengths)
print(testpat)

[]


In [291]:
lcsresults = []
for i in testpat:
    decoded_pat = decode_strings(i)
    print(decoded_pat)
    lcsresults.append(decoded_pat)

In [292]:
def count_substringandparents(allthepatterns, allthepattern_count):
    
    issubstring = {}
    hassubstring = {}
    count_as_substring = {}
    count_as_parent = {}

    for i in range(0, len(allthepatterns)-1):
        for j in range(i+1, len(allthepatterns)):
            if allthepatterns[i] in allthepatterns[j]:
                parent = allthepatterns[j]
                kid = allthepatterns[i]
                times = parent.count(kid)

                if kid not in issubstring:
                    issubstring[kid] = 1
                    count_as_substring[kid] = times * allthepattern_count[parent]
                else:
                    issubstring[kid] += 1
                    count_as_substring[kid] += times * allthepattern_count[parent]

                if parent not in hassubstring:
                    hassubstring[parent] = 1
                    count_as_parent[parent] = 1#times
                else:
                    hassubstring[parent] += 1
                    count_as_parent[parent] += 1 

            elif allthepatterns[j] in allthepatterns[i]:
                parent = allthepatterns[i]
                kid = allthepatterns[j]
                times = parent.count(kid)

                if kid not in issubstring:
                    issubstring[kid] =1
                    count_as_substring[kid] = times * allthepattern_count[parent]
                else:
                    issubstring[kid] += 1
                    count_as_substring[kid] += times * allthepattern_count[parent]

                if parent not in hassubstring:
                    hassubstring[parent] = 1
                    count_as_parent[parent] = 1 #times
                else:
                    hassubstring[parent] += 1
                    count_as_parent[parent] += 1 #times

    return issubstring, hassubstring, count_as_substring, count_as_parent


In [293]:
issubstring, hassubstring, count_as_substring, count_as_parent = count_substringandparents(reducedpat, reduced_pattern_count)

In [294]:
# rank the ones that are substrings of another detected pattern
sortis = dict(Counter(issubstring))
sorthas = dict(Counter(hassubstring))
print(len(sorthas))
print(len(sortis))

5
4


In [295]:
# This counts the times each pattern appeared after original segmentation plus the times they are in other patterns,
# plus the times other patterns showed up in them.

def combined_count(reducedpat, count_as_parent, count_as_substring, reduced_pattern_count):
    refined_combined = {}

    for item in reducedpat:
        refined_combined[item] = reduced_pattern_count[item]
        if item in count_as_parent:
            refined_combined[item] += count_as_parent[item]
        if item in count_as_substring:
            refined_combined[item] += count_as_substring[item]
    
    return refined_combined


In [296]:
combined = combined_count(reducedpat, count_as_parent, count_as_substring, reduced_pattern_count)

In [297]:
#combined

In [298]:
def filter_combined_patterns(combined):
    
    finalpattern = []
    for i in combined: 

        if len(i) < 3:
            # if the length is shorter than 4 notes, discard
            continue

        if combined[i] > 1:
            # keep the ones that has substring or is substring or repeated more than once
            finalpattern.append(i)

    finalpattern_combined = {}
        
    for thisone in finalpattern:
        finalpattern_combined[thisone] = combined[thisone]

    dict(Counter(finalpattern_combined))
        
    if len(finalpattern) > 20:
        finalpattern_combined = dict(sorted(finalpattern_combined.items(), key = lambda x:-x[1], reverse = True)[-20:])
    
    return finalpattern_combined


In [299]:
# filter out the short ones
top20patterns_combined = filter_combined_patterns(combined)

In [300]:
#print(dict(Counter(top20patterns_combined)))

In [301]:
# not filtered by beat strength, top 20
#for printpattern in list(top20patterns_combined):
#    print(decode_strings(printpattern))

In [302]:
#print("the followings are not ranked well thus discarded in the count")
#for i in reducedpat:
#    if i not in top20patterns_combined:
#        print(decode_strings(i))

In [303]:
#finalpat, count_pattern, startonsets,  endonsets = count_final_pattern(reducedfinalpat, strings, beatstrengths)

finalpat, count_pattern, startonsets,  endonsets = count_final_pattern(list(top20patterns_combined), strings, beatstrengths)


Pattern MMGMMFMFMEMIFMEMFMMMM appears in voice no. 1 , first pos starts at  987 end at 1008
discard pattern MMGMMFMFMEMIFMEMFMMMM because it only appered once in total.
Pattern MMMM appears in voice no. 1 , first pos starts at  0 end at 4
Pattern MMMM appears in voice no. 2 , first pos starts at  481 end at 485
Pattern FGFFIGM appears in voice no. 1 , first pos starts at  289 end at 296
discard pattern FGFFIGM because it never appeared at the start of a bar.
this pattern can be decoded as 1D1A1D1D3A1A0A
Pattern FCFMILDI appears in voice no. 1 , first pos starts at  298 end at 306
discard pattern FCFMILDI because it never appeared at the start of a bar.
this pattern can be decoded as 1D4D1D0A3A6A3D3A
Pattern MEM appears in voice no. 1 , first pos starts at  967 end at 970
Pattern MEM appears in voice no. 2 , first pos starts at  15 end at 18
discard pattern MEM because it never appeared at the start of a bar.
this pattern can be decoded as 0A2D0A
Pattern MFMFMEMIFMEMFMJFMC appears in vo

In [304]:
def decode_results(finalpat, count_pattern, combined, beforereduction):
    
    decoded_patterns = []
    decoded_patterns_weighed = {}
    decoded_patterns_count = {}
    decoded_patterns_combined = {}
    
    decode_dia_intervals = {'A': '6D', 'B': '5D', 'C': '4D', 'D': '3D', 'E': '2D', 'F': '1D', 
                            'G': '1A', 'H': '2A', 'I': '3A', 'J': '4A', 'K': '5A', 'L': '6A', 'M': '0A'}

    # decode all the candidates for patterns
    for pattern in finalpat:
        trans_pattern = ""
        for letter in pattern:
            trans = decode_dia_intervals[letter]
            trans_pattern += trans

        decoded_patterns_count[trans_pattern] = count_pattern[pattern]
        
        if pattern in combined:
            # some might be reduced to a shorter pattern
            decoded_patterns_combined[trans_pattern] = combined[pattern]
        else:
            decoded_patterns_combined[trans_pattern] = combined[beforereduction[pattern]]
        decoded_patterns.append(trans_pattern)
        
    return decoded_patterns, decoded_patterns_combined, decoded_patterns_count


Strings of intervals of each voice:

In [305]:
# Strings of intervals of each voice
decoded_strings = []
for string in strings:
    decoded = decode_strings(string)
    decoded_strings.append(decoded)
    print(decoded)
    print("")

0A0A0A0A0A6A0A6D0A0A6A0A6D1A0A4A0A4D0A0A4A0A4D1A0A3A3D1A0A0A4A0A4D1A0A3A3D1A0A0A4A0A4D1A0A3A3D1A0A0A0A1A0A1A0A2A1D1D1D4A3D3A3D2D1A1D1D3A1A0A0A4D1D0A3A6A3D3A3D2D1A1D1D3A1A0A0A4D1D0A3A6A3D3A3D2D1A1D1D3A1A0A4D1D0A5A2A1D0A6D1A0A1D5D5A4A3D3D2D2A0A4D1D5D5A6A2A0A1D5D5A6D0A6A6D0A6A6D0A5A5D0A5A6D0A6A6D0A5A6D0A6A6D0A6A6D0A5A6D0A6A1D1A1D1D1D3D0A1D4A1D5D5A5D0A2A3D3D2D2A0A4D1D5D5A6A2A0A1D5D5A6D0A6A6D0A6A6D0A5A5D0A5A6D0A6A6D0A5A6D0A5A5D0A5A6D0A5A5D0A5A6D0A6A6D0A6A0A3D3A3D3D2A1D1D3A1A0A0A4D1D0A3A6A3D3A3D2D1A1D1D3A1A0A0A4D1D0A3A6A3D3A3D2D1A1D1D3A1A0A4D1D0A5A0A1A0A6D2A1D0A1D1D4A3D3A3D2D1A1D1A1D1D3A1A0A0A1A1D4D1D0A3A6A3D3A3D2D1A1D1A1D1D3A1A0A0A1A1D4D1D0A3A6A3D3A3D2D1A1D1A1D1D3A1A0A4D1D0A5A2A1D0A6D1A0A1D5D5A4A3D3D2D2A0A4D1D5D5A6A2A0A1D5D5A6D0A6A6D0A6A6D0A5A5D0A5A6D0A6A6D0A5A6D0A6A6D0A6A6D0A5A6D0A6A1D1A1D1D1D3D0A1D4A1D5D5A5D0A2A3D3D2D2A0A4D1D5D5A6A2A0A1D5D5A6D0A6A6D0A6A6D0A5A5D0A5A6D0A6A6D0A5A6D0A5A5D0A5A6D0A5A5D0A5A6D0A6A6D0A6A0A3D3A3D3D2A1D1D3A1A0A0A4D1D0A3A6A3D3A3D2D1A1D1D3A1A0A0A4D1D0A3A6A3D3A3D2D1A1D

In [306]:
# Decode patterns
decoded_patterns, decoded_patterns_combined, decoded_patterns_count = decode_results(finalpat, count_pattern, combined, beforereduction)

for pattern in decoded_patterns:
        print("Pattern:", pattern)
        print("importance score:", decoded_patterns_combined[pattern],
              "occurrence:", decoded_patterns_count[pattern])


Pattern: 0A0A0A0A
importance score: 2 occurrence: 7
Pattern: 1A1D1D3A1A0A
importance score: 2 occurrence: 18


In [307]:
# Final patterns(LCS not included), FILTERED BY BEAT STRENGTH:
decoded_patterns

['0A0A0A0A', '1A1D1D3A1A0A']

In [308]:
# THE FINAL LIST
final_results = decoded_patterns+lcsresults

print(final_results)

['0A0A0A0A', '1A1D1D3A1A0A']


Danny's version, ground truth

In [182]:
#groundtruthpath1 = "jkupddtest/gibbonsmono/silverswan.krn"
import pandas as pd
gtfiles = glob.glob("jkupddtest/gibbonsmono/H/midi/*.csv")
print(gtfiles)
gtstrings = []
for gtfile in gtfiles:
    print(gtfile)
    with open(gtfile, 'r') as file:
        df = pd.read_csv(file)
        #print(df)
        #for midi_note_num in df["midi_note_num"]:
        #    print(midi_note_num)
        count = 0
        gtstring = ""
        for diatonic in df["diatonic_interval"]:
            if count == 0:
                # ignore the first "interval" number which is always 0
                count+=1
                continue
            count+=1
            # just to format the input
            if diatonic < 0:
                dia = str(abs(diatonic)) + 'D'
            else:
                dia = str(abs(diatonic)) + 'A'
            gtstring += dia
        gtstrings.append(gtstring)
        print(gtstring)
            

['jkupddtest/gibbonsmono/H/midi/occ1.midi_part1_DD.csv', 'jkupddtest/gibbonsmono/H/midi/occ2.midi_part1_DD.csv']
jkupddtest/gibbonsmono/H/midi/occ1.midi_part1_DD.csv
0A1D1D1D1D0A7A1D1D4D3A1D1A2D1A3D1A0A4D
jkupddtest/gibbonsmono/H/midi/occ2.midi_part1_DD.csv
0A1D1D1D1D0A7A1D1D4D3A1D1A2D1A3D1A0A4D


In [57]:
# a set of strings of annotated ground truth
print(set(gtstrings))

{'0A1D1D1D1D0A7A1D1D4D3A1D1A2D1A3D1A0A4D'}


In [58]:
# check if they are in the scores
for pattern in gtstrings:
    count = 0
    for decodedstring in decoded_strings:
        count += decodedstring.count(pattern)
    if count == 0:
        print("pattern", pattern, "not in original score")
    elif count == 1:
        print("pattern", pattern, "only appeared once")
    else:
        print("pattern", pattern, "appeared", count, "times in total")

pattern 0A1D1D1D1D0A7A1D1D4D3A1D1A2D1A3D1A0A4D not in original score
pattern 0A1D1D1D1D0A7A1D1D4D3A1D1A2D1A3D1A0A4D not in original score


Discarded: get patterns from ground truth folders.. original ground truth

In [57]:
# Get patterns from ground truth

dict_wordtonum = {"Unison": '0', "Second": '1', "Third": '2', "Fourth": '3', "Fifth": '4', "Sixth": '5', "Seventh": '6'}

groundtruthpatterns = []
print(glob.glob("jkupddtest/chopinmono/*"))
filenames = glob.glob("jkupddtest/chopinmono/B/*.csv")
for filename in filenames:
    print(filename)
    with open(filename, 'r') as file:
        csvreader = csv.reader(file)
        gts = []
        gtm = []
        pattern_gt = []
        for row in csvreader:
            gts.append(float(row[0]))
            gtm.append(float(row[1]))
        for i in gtm:
            pattern_gt.append(m21.note.Note(i))
        pattern_string = ""
        interval_string = ""
        for i in pattern_gt:
            pattern_string += str(i)
            
        for i in range(1, len(pattern_gt)):           
            gap = pattern_gt[i].pitch.diatonicNoteNum - pattern_gt[i-1].pitch.diatonicNoteNum
            if gap >= 0:
                direction = 'A'
            else:
                direction = 'D'
            m21_interval_directed = m21.interval.Interval(noteStart=pattern_gt[i-1], noteEnd=pattern_gt[i]).directedSimpleNiceName
            arr_diatonic = m21_interval_directed.split(" ")
            m21_generic = dict_wordtonum[arr_diatonic[-1]]
            m21_interval = m21_generic+direction
            
            interval_string+=m21_interval
            
        print(interval_string)

        groundtruthpatterns.append(interval_string)


['jkupddtest/chopinmono/D', 'jkupddtest/chopinmono/A', 'jkupddtest/chopinmono/mazurka24-4.krn']


In [58]:
set(groundtruthpatterns)

set()

In [59]:
for pattern in groundtruthpatterns:
    count = 0
    for decodedstring in decoded_strings:
        count += decodedstring.count(pattern)
    if count == 0:
        print("pattern", pattern, "not in original score")
    elif count == 1:
        print("pattern", pattern, "only appeared once")
    else:
        print("pattern", pattern, "appeared", count, "times in total")